
**Adopted from:**

https://stackoverflow.com/questions/60877274/optimal-rotation-in-3d-with-kabsch-algorithm



In [22]:
import numpy as np
import csv 

In [23]:
map_points = []
true_points = []

csvfile = 'file.csv'   # enter csv file here with coordinates for the rotation matrix

with open(csvfile, 'r') as f:
    csvreader = csv.reader(f)
    for row in csvreader:
        map_points.append([float(i) for i in row[0:3]])
        true_points.append([float(i) for i in row[3:]])
        

print(map_points)
print(true_points)                 
    

[[0.0945736434108525, 0.873643410852713, 5.0], [-14.3941176470588, -5.55882352941177, 5.0], [-13.2347457627119, 4.6957627118644, 5.0], [-4.61333333333333, 11.9591666666667, 5.0], [4.84344262295082, 12.6114754098361, 5.0], [9.63, 4.5275, 5.0], [12.6933333333333, -5.42666666666667, 5.0], [5.64, -11.9096, 5.0], [-6.32169811320755, -14.8801886792453, 5.0]]
[[0.0, 0.0, 0.0], [0.0, 15.0, 0.0], [12.0, 12.0, 0.0], [15.0, 0.0, 0.0], [12.0, -12.0, 0.0], [0.0, -15.0, 0.0], [-12.0, -12.0, 0.0], [-15.0, 0.0, 0.0], [-12.0, 12.0, 0.0]]


In [24]:
mapped_centroid = np.average(map_points, axis=0)
true_centroid = np.average(true_points, axis=0)

map_points -= mapped_centroid
true_points -= true_centroid


In [25]:
h = map_points.T @ true_points
u, s, vt = np.linalg.svd(h)
v = vt.T

d = np.linalg.det(v @ u.T)
e = np.array([[1, 0, 0], [0, 1, 0], [0, 0, d]])

r = v @ e @ u.T

tt = true_centroid - np.matmul(r, mapped_centroid)
print(r, tt)

[[-0.39324052  0.91943564  0.        ]
 [-0.91943564 -0.39324052  0.        ]
 [ 0.          0.          1.        ]] [ 0.07006845 -0.71427017 -5.        ]


In [26]:
np.save('Rot.npy', r)
np.save('Tr.npy', tt)

**Using scipy** 

In [27]:
from scipy.spatial.transform import Rotation

In [28]:
R = Rotation.align_vectors(map_points, true_points)[0].as_matrix()

In [29]:
print(R)

[[-0.39324052 -0.91943564  0.        ]
 [ 0.91943564 -0.39324052  0.        ]
 [ 0.          0.          1.        ]]


**Reverse testing Rot and tt**

In [37]:
r = [-14.9, -5.55, 5]
r_tran = R @ r + tt
print([round(r_tran[i], 1) for i in range(0, len(r_tran))])

[11.0, -12.2, 0.0]
